In [2]:
import pandas as pd

In [34]:
#dane bez encodingu z poprzedniego etapu
data = pd.read_csv("./out.csv")

## Encoding labelowy

Labelowy, bo nie będzie dużo kolumn co nie spowolni szkolenia modeli.

In [35]:
temp = data.select_dtypes(include=['object'])
temp = temp.astype('category')
for col in temp.columns:
    data[col] = temp[col].cat.codes
    
train_X = data.drop(columns=["readmitted"])
train_y = data["readmitted"]

In [38]:
#zamiana na binarne
data.loc[data['readmitted'] == 0, 'readmitted'] = 1
data.loc[data['readmitted'] == 2, 'readmitted'] = 0
#1 oznacza, ze byl w szpitalu, 0 ze nie wrocil

0    52337
1    45715
Name: readmitted, dtype: int64

In [73]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import scipy.stats
import scipy, skopt

## Domyślne

In [44]:
model0 = GradientBoostingClassifier(random_state=0, verbose=True)
model0.fit(train_X, train_y)

      Iter       Train Loss   Remaining Time 
         1           1.3691           11.84s
         2           1.3587           11.71s
         3           1.3502           11.86s
         4           1.3432           12.06s
         5           1.3374           11.90s
         6           1.3325           11.75s
         7           1.3283           11.55s
         8           1.3247           11.40s
         9           1.3217           11.21s
        10           1.3189           11.22s
        20           1.3002            9.90s
        30           1.2871            8.51s
        40           1.2793            7.23s
        50           1.2736            5.98s
        60           1.2695            4.76s
        70           1.2659            3.56s
        80           1.2631            2.37s
        90           1.2607            1.18s
       100           1.2581            0.00s


GradientBoostingClassifier(random_state=0, verbose=True)

In [45]:
roc_auc_score(train_y, model0.predict(train_X))

0.6284985516852621

In [69]:
accuracy_score(train_y, model0.predict(train_X))

0.6376310529107004

## Ręczne

Szczerze, nie wiem czy jest tutaj wymagane sprawdzenie dużej ilości przypadków, raczej zostawię to na randomsearch oraz na gridsearch.

In [46]:
model1 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0, verbose=True)
model1.fit(train_X, train_y)

      Iter       Train Loss   Remaining Time 
         1           1.3366            4.45s
         2           1.3303            4.45s
         3           1.3242            4.35s
         4           1.3184            4.33s
         5           1.3141            4.27s
         6           1.3116            4.19s
         7           1.3091            4.13s
         8           1.3072            4.10s
         9           1.3040            4.04s
        10           1.3014            3.99s
        20           1.2897            3.57s
        30           1.2839            3.13s
        40           1.2794            2.67s
        50           1.2765            2.26s
        60           1.2740            1.80s
        70           1.2720            1.35s
        80           1.2700            0.90s
        90           1.2687            0.45s
       100           1.2677            0.00s


GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=0,
                           verbose=True)

In [47]:
roc_auc_score(train_y, model1.predict(train_X))

0.6238769904506942

In [70]:
accuracy_score(train_y, model1.predict(train_X))

0.6326235058948313

## Random search

In [64]:
parameters = {'loss':('deviance', 'exponential'), 'learning_rate':scipy.stats.expon(scale=.1),
              'n_estimators':[100]}
model_temp = GradientBoostingClassifier()
RS = RandomizedSearchCV(model_temp, parameters, scoring = 'roc_auc', return_train_score=True, verbose=3)
RS.fit(train_X, train_y)
RS.best_params_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END learning_rate=0.035293535081177003, loss=deviance, n_estimators=100;, score=(train=0.681, test=0.657) total time=   8.8s
[CV 2/5] END learning_rate=0.035293535081177003, loss=deviance, n_estimators=100;, score=(train=0.680, test=0.668) total time=   9.0s
[CV 3/5] END learning_rate=0.035293535081177003, loss=deviance, n_estimators=100;, score=(train=0.683, test=0.657) total time=   8.9s
[CV 4/5] END learning_rate=0.035293535081177003, loss=deviance, n_estimators=100;, score=(train=0.677, test=0.678) total time=   8.9s
[CV 5/5] END learning_rate=0.035293535081177003, loss=deviance, n_estimators=100;, score=(train=0.677, test=0.684) total time=   8.9s
[CV 1/5] END learning_rate=0.11514015384658649, loss=exponential, n_estimators=100;, score=(train=0.699, test=0.668) total time=   8.8s
[CV 2/5] END learning_rate=0.11514015384658649, loss=exponential, n_estimators=100;, score=(train=0.696, test=0.673) total time=   8.

{'learning_rate': 0.3214279965465431,
 'loss': 'exponential',
 'n_estimators': 100}

In [65]:
model_RS = GradientBoostingClassifier(learning_rate=0.3214279965465431, loss='exponential', n_estimators=100)
model_RS.fit(train_X, train_y)

GradientBoostingClassifier(learning_rate=0.3214279965465431, loss='exponential')

In [66]:
roc_auc_score(train_y, model_RS.predict(train_X))

0.6425857235294186

In [71]:
accuracy_score(train_y, model_RS.predict(train_X))

0.6490841594256109

## Grid search

In [55]:
parameters = {'loss':('deviance', 'exponential'), 'learning_rate':[0.01, 0.1, 1.0],
              'n_estimators':[10, 100, 200], }
model_temp = GradientBoostingClassifier()
GS = GridSearchCV(model_temp, parameters, scoring = 'roc_auc', return_train_score=True, verbose=3)
GS.fit(train_X, train_y)
GS.best_params_

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END learning_rate=0.01, loss=deviance, n_estimators=10;, score=(train=0.642, test=0.622) total time=   1.0s
[CV 2/5] END learning_rate=0.01, loss=deviance, n_estimators=10;, score=(train=0.642, test=0.648) total time=   1.1s
[CV 3/5] END learning_rate=0.01, loss=deviance, n_estimators=10;, score=(train=0.647, test=0.629) total time=   1.0s
[CV 4/5] END learning_rate=0.01, loss=deviance, n_estimators=10;, score=(train=0.643, test=0.648) total time=   1.0s
[CV 5/5] END learning_rate=0.01, loss=deviance, n_estimators=10;, score=(train=0.642, test=0.655) total time=   1.0s
[CV 1/5] END learning_rate=0.01, loss=deviance, n_estimators=100;, score=(train=0.658, test=0.640) total time=  10.8s
[CV 2/5] END learning_rate=0.01, loss=deviance, n_estimators=100;, score=(train=0.656, test=0.654) total time=  10.6s
[CV 3/5] END learning_rate=0.01, loss=deviance, n_estimators=100;, score=(train=0.660, test=0.635) total time=  10.5s


[CV 5/5] END learning_rate=1.0, loss=deviance, n_estimators=100;, score=(train=0.729, test=0.682) total time=  10.3s
[CV 1/5] END learning_rate=1.0, loss=deviance, n_estimators=200;, score=(train=0.736, test=0.652) total time=  21.2s
[CV 2/5] END learning_rate=1.0, loss=deviance, n_estimators=200;, score=(train=0.748, test=0.666) total time=  21.2s
[CV 3/5] END learning_rate=1.0, loss=deviance, n_estimators=200;, score=(train=0.751, test=0.659) total time=  20.6s
[CV 4/5] END learning_rate=1.0, loss=deviance, n_estimators=200;, score=(train=0.748, test=0.686) total time=  21.0s
[CV 5/5] END learning_rate=1.0, loss=deviance, n_estimators=200;, score=(train=0.749, test=0.674) total time=  20.4s
[CV 1/5] END learning_rate=1.0, loss=exponential, n_estimators=10;, score=(train=0.689, test=0.655) total time=   1.0s
[CV 2/5] END learning_rate=1.0, loss=exponential, n_estimators=10;, score=(train=0.683, test=0.667) total time=   1.0s
[CV 3/5] END learning_rate=1.0, loss=exponential, n_estimato

{'learning_rate': 0.1, 'loss': 'deviance', 'n_estimators': 200}

In [59]:
model_GS = GradientBoostingClassifier(learning_rate=0.1, loss='deviance', n_estimators=200)
model_GS.fit(train_X, train_y)

GradientBoostingClassifier(n_estimators=200)

In [60]:
roc_auc_score(train_y, model_GS.predict(train_X))

0.6371257865634341

In [72]:
accuracy_score(train_y, model_GS.predict(train_X))

0.6445865459144128

## Optymalizacja bayesowska

In [75]:
search_spaces = {
        'loss': ['deviance', 'exponential'],
        'n_estimators': skopt.space.space.Integer(1, 1000),
        'learning_rate': skopt.space.space.Real(0, 2)
}

In [77]:
model_temp = GradientBoostingClassifier()
opt_1 = skopt.BayesSearchCV(
        estimator=model_temp,
        search_spaces=search_spaces,
        scoring='roc_auc',
        n_jobs=4,
        n_iter=10,
        verbose=3,
        random_state=1
    )
opt_1.fit(train_X, train_y)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits


BayesSearchCV(estimator=GradientBoostingClassifier(), n_iter=10, n_jobs=4,
              random_state=1, scoring='roc_auc',
              search_spaces={'learning_rate': Real(low=0, high=2, prior='uniform', transform='normalize'),
                             'loss': ['deviance', 'exponential'],
                             'n_estimators': Integer(low=1, high=1000, prior='uniform', transform='normalize')},
              verbose=3)

In [78]:
opt_1.best_params_

OrderedDict([('learning_rate', 0.23295552444912887),
             ('loss', 'exponential'),
             ('n_estimators', 626)])

In [79]:
model_OB = GradientBoostingClassifier(learning_rate = 0.23295552444912887, loss = 'exponential', n_estimators = 626)
model_OB.fit(train_X, train_y)

GradientBoostingClassifier(learning_rate=0.23295552444912887,
                           loss='exponential', n_estimators=626)

In [80]:
roc_auc_score(train_y, model_OB.predict(train_X))

0.6681935799042171

In [81]:
accuracy_score(train_y, model_OB.predict(train_X))

0.6728980540937461